### churn prediction
- predict if a customer will exit the bank

In [2]:
#### pre-requisites
import warnings
warnings.filterwarnings(action="ignore")

In [3]:
#### import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#### load the data
df = pd.read_csv('./Churn_Modelling.csv')
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


#### exploratory data analysis

In [6]:
# get the general information about the df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
# check for missing records
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
# check the exited column
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

#### data cleansing

In [9]:
# remove the unwanted columns
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [10]:
from sklearn.preprocessing import LabelEncoder

# convert the non-numerical columns to numeric ones
df['Geography'] = LabelEncoder().fit_transform(df['Geography'])
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])

In [11]:
# correlation analysis
df.corr()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,0.007888,-0.002857,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Geography,0.007888,1.000000,0.004719,0.022812,0.003739,0.069408,0.003972,-0.008523,0.006724,-0.001369,0.035943
Gender,-0.002857,0.004719,1.000000,-0.027544,0.014733,0.012087,-0.021859,0.005766,0.022544,-0.008112,-0.106512
Age,-0.003965,0.022812,-0.027544,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,0.000842,0.003739,0.014733,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,0.006268,0.069408,0.012087,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,0.003972,-0.021859,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,-0.008523,0.005766,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,0.006724,0.022544,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,-0.001369,-0.008112,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [12]:
# since the domain knowledge says the EstimatedSalary is not significant, lets remove it
df.drop('EstimatedSalary', axis=1, inplace=True)

#### split the data

In [13]:
# create x
x = df.drop('Exited', axis=1)

# create y
y = df['Exited']

In [14]:
from sklearn.model_selection import train_test_split

# split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, random_state=123456
)

#### 

model building

In [15]:
from sklearn.linear_model import LogisticRegressionCV

def create_logistic_regression_model():

    # create model
    model = LogisticRegressionCV(max_iter=1000)

    # train the model
    model.fit(x_train, y_train)

    return model

In [16]:
from sklearn.ensemble import RandomForestClassifier

def create_random_forest_model():

    # create a model
    model = RandomForestClassifier(n_estimators=60, max_depth=10)
    
    # train the model
    model.fit(x_train, y_train)

    return model

In [17]:
from sklearn.neighbors import KNeighborsClassifier

def create_knn_model():

    # create the model
    model = KNeighborsClassifier(n_neighbors=5)

    # train the model
    model.fit(x_train, y_train)

    return model

In [18]:
from sklearn.svm import SVC

def create_svm_model():

    # create the model
    model = SVC(C=2, kernel='rbf')

    # train the model
    model.fit(x_train, y_train)

    return model

In [19]:
from sklearn.naive_bayes import GaussianNB

def create_naive_bayes_model():

    # create the model
    model = GaussianNB()

    # train the model
    model.fit(x_train, y_train)

    return model

In [20]:
from sklearn.tree import DecisionTreeClassifier

def create_decision_tree_model():

    # create the model
    model = DecisionTreeClassifier(max_depth=10)

    # train the model
    model.fit(x_train, y_train)

    return model

#### model evaluation

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model):
    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return f"{accuracy:.2f}", f"{precision:.2f}", f"{recall:.2f}", f"{f1:.2f}"

In [22]:
# create all the models
models = []
models.append({"name": "Logistic Regression", "model": create_logistic_regression_model()})
models.append({"name": "KNN", "model": create_knn_model()})
models.append({"name": "SVM", "model": create_svm_model()})
models.append({"name": "Naive Bayes", "model": create_naive_bayes_model()})
models.append({"name": "Decision Tree", "model": create_decision_tree_model()})
models.append({"name": "Random Forest", "model": create_random_forest_model()})

In [23]:
# evaluate all the models
for info in models:
    accuracy, precision, recall, f1 = evaluate_model(info["model"])
    info["accuracy"] = accuracy
    info["precision"] = precision
    info["recall"] = recall
    info["f1"] = f1

In [24]:
# print the evaluation report
df_evaluation_report = pd.DataFrame(models, columns=["name", "accuracy", "precision", "recall", "f1"])
df_evaluation_report

,name,accuracy,precision,recall,f1
0,Logistic Regression,0.80,0.58,0.14,0.23
1,KNN,0.75,0.22,0.09,0.13
2,SVM,0.79,0.00,0.00,0.00
3,Naive Bayes,0.79,0.43,0.08,0.13
4,Decision Tree,0.83,0.61,0.45,0.52
5,Random Forest,0.85,0.79,0.38,0.52


In [ ]:
# test data
# can you use the boosting techniques --> adaboost/catboost/XGboost

# train data
# also generate all models
# can you use the boosting techniques --> adaboost/catboost/XGboost

# visualize the data